# Transformacion de los esquemas JSON.

In [136]:
#Peticiones
#importamos la libreria para eralizar peticiones al servidor
import pandas as pd
import numpy as np
import re
import requests as req

url_personas = "https://wstest.paulsaenzsucre.dev/people"
url_proyectos = "https://wstest.paulsaenzsucre.dev/projects"
url_asistencias = "https://wstest.paulsaenzsucre.dev/attendances"

personas = req.get(url_personas)
proyectos = req.get(url_proyectos)
asistencias = req.get(url_asistencias)



if personas.status_code == 200 | proyectos.status_code == 200 | asistencias.status_code == 200:

    print("Peticiones realizadas con éxito")
else: 
    print(f"Error {personas.status_code} & {proyectos.status_code} & {asistencias.status_code}")


Peticiones realizadas con éxito


In [137]:
from pprint import pprint
#Para los objetos personas
df_personas = pd.DataFrame(personas.json())


#Para los objetos asistencias
df_asistencias = pd.DataFrame(asistencias.json())

#Para el objeto proyectos lo convertimos en cadena para tratarlo y separamos los proyectos individuales
list_proyectos = str(proyectos.json()).split('}, {')
print(list_proyectos)


["[{'id': 1, 'code': 'Ob219932', 'name': 'Proyecto 01', 'location': {'latitude': -12.1758997, 'longitude': -76.945643, 'name': 'La Ruta', 'address': 'Av. Los Palos 234.'}, 'radius': 50", "'id': 2, 'code': 'Ob219933', 'name': 'Proyecto 02', 'location': {'latitude': 20.552370071411, 'longitude': -100.38117980957, 'name': 'La Ruta', 'address': 'Av. Los Palos 234.'}, 'radius': 50}]"]


In [204]:
import json
import ast

str_locaciones = []
str_proyectos = []
for p in range(0,len(list_proyectos)):
    str_locaciones.append(list_proyectos[p][re.search("\{'latitude':" ,list_proyectos[p]).span()[0] : re.search("'radius': \d+",list_proyectos[p]).span()[1]] + '}')
    str_proyectos.append(list_proyectos[p].replace(str_locaciones[p].strip('}'), "''"))

#realizamos una pequeña limpieza de los datos.
for l in range(len(str_locaciones)):
    str_locaciones[l] = str_locaciones[l].replace('}, ' , ',')

#Para los extremos
str_proyectos[0] = str_proyectos[0].replace('[' , '') + '}'
str_proyectos[-1] = str_proyectos[-1].replace(']','')

#Para los medios
for l in range(1,len(str_proyectos)):
    str_proyectos[l] = '{' + str_proyectos[l]

#Convertimos las cadenas en diccionarios
dict_locaciones = [ast.literal_eval(cadena) for cadena in str_locaciones]
dict_proyectos = [ast.literal_eval(cadena) for cadena in str_proyectos]

#Convertimos los diccionarios en dataframes
df_locaciones = pd.DataFrame(dict_locaciones)
df_proyectos = pd.DataFrame(dict_proyectos)

#Juntamos la información
proyectos = pd.concat([df_proyectos,df_locaciones], axis = 1).drop(columns=['location'])
proyectos


,id,code,name,latitude,longitude,name,address,radius
0,1,Ob219932,Proyecto 01,-12.17590,-76.945643,La Ruta,Av. Los Palos 234.,50
1,2,Ob219933,Proyecto 02,20.55237,-100.381180,La Ruta,Av. Los Palos 234.,50


In [6]:
#Tratamos el archivo JSON de forma total.

#importamos el documento y lo convertimos a cadena
#crudo = open("est.txt","r+")
#prueba = crudo.read()
prueba = str(personas.json())



#separamos el JSON por estructuras de datos (peticiones)
i=0 ;j=0
lista = []
while i != -1:
  lista.append(prueba[i+1:j])
  i=j ; i = prueba.find('[',i) ; j = prueba.find(']',i)

r = lista[1:]

#Tenemos que eliminar saltos de linea y posteriormente convertirlo a diccionario.
r = [i.replace('\n','') for i in r]
r = [i.replace('"location": {','') for i in r]

#extraemos los radios de ubicacion:
radios = pd.DataFrame({'radius' : re.findall(r'"radius":\s*(\d+)',r[1])})
#Separamos las estructuras de datos por empleado
l = [re.findall(r'\{.*?\}', cadena) for cadena in r]

#Dividimos la información por concepto de dato
personal = [l[0][j] for j in range(len(l[0][:]))]
proyectos = [l[1][j] for j in range(len(l[1][:]))]
asistencias = [l[2][j] for j in range(len(l[2][:]))]

#Convertimos en dataframe la información, donde cada registro sea la informacion de un trabajador
df_personal = pd.DataFrame([eval(componente) for componente in personal])
df_proyectos = pd.DataFrame([eval(componente) for componente in proyectos])
df_asistencias = pd.DataFrame([eval(componente) for componente in asistencias])

#agregamos la columna radius a la estructura proyectos
df_proyectos['radius'] = pd.DataFrame(re.findall(r'"radius":\s*(\d+)',r[1]))
#unimos las tres tablas, cambiamos nombres y ordenamos las columnas
df = pd.merge(df_personal,df_asistencias, on='id', how='inner')[['id','name','phone','projectId','checkin','checkout']]
asistencias = pd.merge(df,df_proyectos,on='id',how='inner')
asistencias.rename(columns = {'name_x':'employee' , 'name_y':'proyect','code':'po'},inplace = True)
asistencias = asistencias[['id','employee','phone','checkin','checkout','proyect','po','latitude','longitude','radius','address']]


IndexError: list index out of range

In [ ]:
df1 = pd.DataFrame(proyectos.json())

df1

,id,code,name,location,radius
0,1,Ob219932,Proyecto 01,"{'latitude': -12.1758997, 'longitude': -76.945...",50
1,2,Ob219933,Proyecto 02,"{'latitude': 20.552370071411, 'longitude': -10...",50


In [ ]:
df3 = pd.DataFrame(asistencias.json())

df3

,id,personId,checkin
0,1,1,2024-10-29T15:41:44.000Z
